# <center> Segmenting and Clustering Toronto Neighbourhoods <center>

# Objective: Segment and cluster the Toronto neighbourhoods based on post codes

Download the table of post codes for neighbourhoods in Toronto from Wikipedia. https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

I have Excel 2016, which can hold 1,048,576 rows. The post code table has only 287 rows. I tried using BeautifulSoup first. But when I was half-way through and dealing with all those issues of cleaning the data, I realized that for such a small table, directlying downloading it into Excel, then uploading it into Jupyter Notebook will be much easier and faster as well. So, I scraped all my BeautifulSoup codes.

In [1]:
import pandas as pd
postcode_df=pd.read_excel('Toronto Post Codes.xlsx')
postcode_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [2]:
# size of the table
postcode_df.shape

(287, 3)

In [3]:
# number of unique Boroughs
import numpy as np
postcode_df["Borough"].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

Okay. So there are 77 "Not assigned" in the Borough column. I could use drop.duplicates() to drop those "Not assigned". However, that will drop the other boroughs as well, since there are boroughs that have more than one neighbourhoods assigned to it. So, drop by "Borough" won't work.

Let's check the Neighbourhood column. The neighbourhoods should be unique, except for the "Not assigned".

In [4]:
postcode_df["Neighbourhood"].value_counts()

Not assigned         77
St. James Town        2
Runnymede             2
The Danforth West     1
Cliffcrest            1
                     ..
York University       1
Weston                1
Mimico South          1
Golden Mile           1
High Park             1
Name: Neighbourhood, Length: 209, dtype: int64

All the neighbourhoods are unique, except for 1) the "Not assigned" and  2) "Runnymede" and "St. James Town" where there are two copies.

In [5]:
#Remove the "Not assigned" from the Borough column.
postcode_df.drop_duplicates(subset="Neighbourhood",keep=False, inplace=True)
postcode_df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


All the "Not assigned" in the Neighbourhood column has been dropped. However, since the parameter "keep" was set to False. The process above dropped the two neighbourhoods "Runnemede" and "St. James Town" as well. But I would like to keep those two neighbourhoods, since each copy belong to two different boroughs.

Put the two neighbourhoods back.

In [6]:
append1=pd.DataFrame({"Postcode":["M5C","M6N","M6S","M4X"],"Borough":["Downtown Toronto","York","West Toronto","Downtown Toronto"],"Neighbourhood":["St. James Town","Runnymede","Runnymede","St. James Town"]})
postcode_df.append(append1,ignore_index=False)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
285,M8Z,Etobicoke,South of Bloor
0,M5C,Downtown Toronto,St. James Town
1,M6N,York,Runnymede
2,M6S,West Toronto,Runnymede


As shown above,the original table has 287 rows in total and 77 "Not assigned" in the Borough column. After the 77 "Not assigned" rows are dropped, the cleaned table should have 287-77=210 rows. The result of the above code block shows 210 rows. 

Let's confirm that there is no more "Not assigned" in Borough column.

In [7]:
postcode_df["Borough"].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    35
Central Toronto     17
West Toronto        12
York                 8
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

Cool. There is no more "Not assigned" in the Borough column. 

The table is cleaned now. I can start working with it. The next thing I am going to do is to put the neighbourhoods that have the same postcode in the same row. Let's see how many unique postcodes are in the cleaned table.

In [8]:
postcode_df["Postcode"].value_counts()

M8Y    8
M9V    8
M5V    7
M4V    5
M9B    5
      ..
M3M    1
M1G    1
M4C    1
M1S    1
M2H    1
Name: Postcode, Length: 102, dtype: int64

There are 102 unique postcodes. Most of the postcodes has only 1 neighbourhoods associated with it. However, postcodeS "M8Y" and "M9V" have 8 neighbourhoods associated with it, "M5V" has 7, "M4V" and "M8Z" have 5. I need to concatenate all those neighbourhoods under the postcode they are associated with, in one row.

In [9]:
postcode_df=pd.DataFrame(postcode_df.groupby(["Postcode","Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)))
postcode_df

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."


All the neighbourhoods are now concatenated under the postcode they are associated with. However, instead of regular index, the table has "Postcode" and "Borough" as its multi-column index. I am going to fix it below.

In [10]:
postcode_df.reset_index(inplace=True)
postcode_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
97,M9N,York,Weston
98,M9P,Etobicoke,Westmount
99,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
100,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [11]:
print(postcode_df.shape)

(102, 3)


In [16]:
# Load the "Geospatial_Coordinates.csv" dataset
Geo_df=pd.read_csv("Geospatial_Coordinates.csv")
Geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
# Merge "postcode_df" and "Geo_df". 
# Use "left" join so that all records from "postcode_df" and matched records from "Geo_df" are returned.
Geomerge_df=pd.merge(postcode_df,Geo_df, how="left")
Geomerge_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
97,M9N,York,Weston,43.706876,-79.518188
98,M9P,Etobicoke,Westmount,43.696319,-79.532242
99,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
100,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
